In [3]:
pip install cryptography

   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
    --------------------------------------- 0.0/3.1 MB 487.6 kB/s eta 0:00:07
   --- ------------------------------------ 0.3/3.1 MB 2.5 MB/s eta 0:00:02
   ------------ --------------------------- 0.9/3.1 MB 5.9 MB/s eta 0:00:01
   --------------------- ------------------ 1.6/3.1 MB 7.9 MB/s eta 0:00:01
   ---------------------------- ----------- 2.2/3.1 MB 8.7 MB/s eta 0:00:01
   ------------------------------------ --- 2.8/3.1 MB 9.4 MB/s eta 0:00:01
   ---------------------------------------- 3.1/3.1 MB 8.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
from cryptography.fernet import Fernet
from werkzeug.utils import secure_filename
from zipfile import ZipFile

# Set up directories for upload and download
UPLOAD_FOLDER = 'uploads'
DOWNLOAD_FOLDER = 'downloads'

if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

if not os.path.exists(DOWNLOAD_FOLDER):
    os.makedirs(DOWNLOAD_FOLDER)

# Function to generate encryption key
def generate_key():
    key = Fernet.generate_key()
    with open('secret.key', 'wb') as key_file:
        key_file.write(key)

# Function to load an existing key
def load_key(key_path):
    return open(key_path, 'rb').read()

# File encryption function
def encrypt_file(file_path):
    # Generate a new encryption key and save it
    key = Fernet.generate_key()
    with open('key.key', 'wb') as key_file:
        key_file.write(key)
    
    # Create a Fernet instance with the generated key
    fernet = Fernet(key)
    
    # Read the file to be encrypted
    with open(file_path, 'rb') as original_file:
        original = original_file.read()
    
    # Encrypt the file content
    encrypted = fernet.encrypt(original)
    
    # Write the encrypted content to a new file
    encrypted_file_path = file_path.split('.')[0] + '_encrypted' + '.' + file_path.split('.')[-1]
    with open(encrypted_file_path, 'wb') as encrypted_file:
        encrypted_file.write(encrypted)
    
    # Create a zip file containing the encrypted file and the key
    zip_path = os.path.join(UPLOAD_FOLDER, 'encrypted_file.zip')
    with ZipFile(zip_path, 'w') as zip_file:
        zip_file.write(encrypted_file_path)
        zip_file.write('key.key')

    # Clean up the intermediate files
    os.remove(encrypted_file_path)
    os.remove('key.key')

    return zip_path


# File decryption function
def decrypt_file(encrypted_file_path, key_file_path):
    # Load the encryption key
    key = load_key(key_file_path)
    fernet = Fernet(key)

    # Read the encrypted file
    with open(encrypted_file_path, 'rb') as encrypted_file:
        encrypted = encrypted_file.read()
    
    # Decrypt the content
    decrypted = fernet.decrypt(encrypted)
    
    # Write the decrypted content to a new file
    decrypted_file_path = os.path.join(DOWNLOAD_FOLDER, 'decrypted_file.txt')
    with open(decrypted_file_path, 'wb') as decrypted_file:
        decrypted_file.write(decrypted)

    return decrypted_file_path


# Example usage
file_path = 'sample.txt'  # Replace with your file path

# Encrypt the file and get the path of the zip containing encrypted file and key
zip_path = encrypt_file(file_path)
print(f"Encrypted file and key saved at: {zip_path}")

# Decrypt the file (assuming you have the zip file and key)
decrypted_file_path = decrypt_file('encrypted_file.zip', 'key.key')
print(f"Decrypted file saved at: {decrypted_file_path}")


FileNotFoundError: [Errno 2] No such file or directory: 'sample.txt'